# اتصال به گوکل کولب

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# خواندن اطلاعات

In [2]:
import pandas as pd
df = pd.read_excel (r'/content/drive/My Drive/DataSet/Spam Base/Data.xlsx')
Data = df.values.tolist()

1 Group

In [11]:
import pandas as pd
Target=[]
df = pd.read_excel (r'/content/drive/My Drive/DataSet/Spam Base/Target.xlsx')
Group_1 = df.values.tolist()
for i in range(0,len(Group_1)):
 Target.append(Group_1[i][0])

# تقسیم بندی داده ها

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(Data, Target, test_size=0.3, random_state=42)

# پیش پردازش

In [14]:
from tensorflow.keras.utils import to_categorical
trainy = to_categorical(y_train)
testy = to_categorical(y_test)

In [15]:
import numpy as np
trainX=[]
for i in range(0,len(X_train)):
  x=np.array(X_train[i])
  trainX.append(np.tile(x,(10,1)))

In [16]:
testX=[]
for i in range(0,len(X_test)):
  x=np.array(X_test[i])
  testX.append(np.tile(x,(10,1)))

In [17]:
trainX=np.array(trainX)
testX=np.array(testX)
trainX=np.reshape(trainX,(len(trainX),10,57,1))
testX=np.reshape(testX,(len(testX),10,57,1))

# استخراج ویژگی

In [52]:
from keras import layers
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D,SeparableConv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers import TimeDistributed
from keras.models import Model

myInput=layers.Input(shape=(10,57,1))
conv1=Conv2D(64,3, activation='relu',padding='same')(myInput)
max1=MaxPooling2D(2)(conv1)
conv2=Conv2D(64,3, activation='relu',padding='same')(max1)
max2=MaxPooling2D(2)(conv2)
flat=Flatten()(max2)
des1 =Dense(500, activation='relu')(flat)
des4 =Dense(50, activation='relu')(des1)
Extract_Model = Model(myInput, des4)

out_layer =Dense(2, activation='softmax')(des4)   # 0  1  Class 
model = Model(myInput, out_layer)
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 10, 57, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 57, 64)        640       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 5, 28, 64)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 5, 28, 64)         36928     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 2, 14, 64)        0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 1792)              0   

In [53]:
verbose, epochs, batch_size = 1, 20, 32
model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)

Epoch 1/20
101/101 [==============================] - 5s 49ms/step - loss: 0.7459 - accuracy: 0.8189
Epoch 2/20
101/101 [==============================] - 5s 49ms/step - loss: 0.2513 - accuracy: 0.9115
Epoch 3/20
101/101 [==============================] - 5s 49ms/step - loss: 0.2095 - accuracy: 0.9258
Epoch 4/20
101/101 [==============================] - 5s 49ms/step - loss: 0.2013 - accuracy: 0.9304
Epoch 5/20
101/101 [==============================] - 5s 49ms/step - loss: 0.1635 - accuracy: 0.9444
Epoch 6/20
101/101 [==============================] - 5s 49ms/step - loss: 0.1585 - accuracy: 0.9450
Epoch 7/20
101/101 [==============================] - 5s 49ms/step - loss: 0.1515 - accuracy: 0.9453
Epoch 8/20
101/101 [==============================] - 5s 50ms/step - loss: 0.1334 - accuracy: 0.9512
Epoch 9/20
101/101 [==============================] - 5s 54ms/step - loss: 0.1380 - accuracy: 0.9488
Epoch 10/20
101/101 [==============================] - 5s 49ms/step - loss: 0.1259 - accura

# طبقه بندی

In [54]:
trainFeature=(Extract_Model.predict(trainX))
testFeature=(Extract_Model.predict(testX))

44/44 [==============================] - 1s 14ms/step


In [55]:
from sklearn import svm
classifier_svm = svm.LinearSVC()
classifier_svm.fit(trainFeature, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC()

In [56]:
from sklearn import tree
classifier_tree = tree.DecisionTreeClassifier()
classifier_tree.fit(trainFeature, y_train)

DecisionTreeClassifier()

In [57]:
from sklearn.neighbors import KNeighborsClassifier
classifier_Knn = KNeighborsClassifier(n_neighbors=3)
classifier_Knn.fit(trainFeature, y_train)

KNeighborsClassifier(n_neighbors=3)

# ترکیب طبقه بند

In [58]:
from sklearn.ensemble import  VotingClassifier
classifier_Voting = VotingClassifier(estimators=[('SVM', classifier_svm), ('TREE', classifier_tree), ('KNN', classifier_Knn)],weights=[1,1,1])
classifier_Voting.fit(trainFeature, y_train)
Class_Voting=classifier_Voting.predict(testFeature)

/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# ارزیابی

In [59]:
from sklearn.metrics import classification_report, accuracy_score
print(classification_report(y_test,Class_Voting))

              precision    recall  f1-score   support

           0       0.94      0.97      0.96       803
           1       0.95      0.92      0.94       577

    accuracy                           0.95      1380
   macro avg       0.95      0.94      0.95      1380
weighted avg       0.95      0.95      0.95      1380



In [60]:
print('Accuracy= ' + str(round(accuracy_score(y_test, Class_Voting)*100))) 

Accuracy= 95


In [61]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, Class_Voting)

array([[778,  25],
       [ 48, 529]])